In [57]:
import pandas as pd
import sqlalchemy
import pymysql
from urllib.parse import quote 
from sqlalchemy import create_engine
import pymysql




In [58]:
!pip install geopy


In [59]:
df = pd.read_csv('data.csv')

df.columns


Index(['id', 'speech', 'display_as', 'party', 'constituency', 'mnis_id',
       'date', 'time', 'colnum', 'speech_class', 'major_heading',
       'minor_heading', 'oral_heading', 'year', 'hansard_membership_id',
       'speakerid', 'person_id', 'speakername', 'url', 'speech_cleaned'],
      dtype='object')

In [60]:
df = df[['id', 'speech_cleaned', 'party', 'constituency', 'display_as']]
df

,id,speech_cleaned,party,constituency,display_as
0,uk.org.publicwhip/debate/2021-01-11c.1.6,what steps he is taking to support leaseholder...,Independent,Romsey and Southampton North,Caroline Nokes
1,uk.org.publicwhip/debate/2021-01-11c.1.7,"in beginning, may i wish you, mr speaker, all ...",Conservative,Tamworth,Christopher Pincher
2,uk.org.publicwhip/debate/2021-01-11c.1.8,sleep deprivation is recognised as a form of t...,Independent,Romsey and Southampton North,Caroline Nokes
3,uk.org.publicwhip/debate/2021-01-11c.1.9,i am grateful to my right hon. friend; she has...,Conservative,Tamworth,Christopher Pincher
4,uk.org.publicwhip/debate/2021-01-11c.1.11,what plans the government have to increase fun...,Conservative,Bosworth,Luke Evans
...,...,...,...,...,...
16200,uk.org.publicwhip/debate/2021-04-29a.519.1,"on a point of order, mr speaker. as a further ...",Conservative,Haltemprice and Howden,David Davis
16201,uk.org.publicwhip/debate/2021-04-29a.519.2,i totally agree with everything that the right...,Speaker,Chorley,Lindsay Hoyle
16202,uk.org.publicwhip/debate/2021-04-29a.519.6,i have to acquaint the house that the house ha...,Speaker,Chorley,Lindsay Hoyle
16203,uk.org.publicwhip/debate/2021-04-29a.521.1,i have further to acquaint the house that the ...,Speaker,Chorley,Lindsay Hoyle


In [61]:
# read csv constituency data
df_constituency = pd.read_csv('constitu.csv')
df_constituency = df_constituency[['PCON21NM', 'LAT', 'LONG']]

# rename lat with latitude and long with longitude
df_constituency.rename(columns={'PCON21NM': 'constituency', 'LAT': 'latitude', 'LONG': 'longitude'}, inplace=True)
df_constituency

# merge df and df_constituency
df = pd.merge(df, df_constituency, on='constituency')

# rename display_as with name
df.rename(columns={'display_as': 'name'}, inplace=True)
df

,id,speech_cleaned,party,constituency,name,latitude,longitude
0,uk.org.publicwhip/debate/2021-01-11c.1.6,what steps he is taking to support leaseholder...,Independent,Romsey and Southampton North,Caroline Nokes,51.05168,-1.487280
1,uk.org.publicwhip/debate/2021-01-11c.1.8,sleep deprivation is recognised as a form of t...,Independent,Romsey and Southampton North,Caroline Nokes,51.05168,-1.487280
2,uk.org.publicwhip/debate/2021-01-11c.46.5,retraining opportunities will be a crucial par...,Independent,Romsey and Southampton North,Caroline Nokes,51.05168,-1.487280
3,uk.org.publicwhip/debate/2021-01-13b.278.6,it is crucial that the vaccine confidence camp...,Independent,Romsey and Southampton North,Caroline Nokes,51.05168,-1.487280
4,uk.org.publicwhip/debate/2021-01-14b.535.4,can i ask my hon. friend what specific measure...,Independent,Romsey and Southampton North,Caroline Nokes,51.05168,-1.487280
...,...,...,...,...,...,...,...
16180,uk.org.publicwhip/debate/2021-04-27c.287.2,i am grateful for the hon. member’s question. ...,Conservative,Aldershot,Leo Docherty,51.28895,-0.784100
16181,uk.org.publicwhip/debate/2021-04-27c.300.0,i am grateful for all members’ contributions. ...,Conservative,Aldershot,Leo Docherty,51.28895,-0.784100
16182,uk.org.publicwhip/debate/2021-04-22a.1147.2,as we move along the road map towards normalit...,Conservative,East Yorkshire,Greg Knight,53.98722,-0.562360
16183,uk.org.publicwhip/debate/2021-04-27c.228.3,what fiscal steps his department is taking to ...,Conservative,Sittingbourne and Sheppey,Gordon Henderson,51.35064,0.683769


In [62]:
# show rows with null values
df[df.isnull().any(axis=1)]

,id,speech_cleaned,party,constituency,name,latitude,longitude


In [63]:
# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut

# # Initialize geolocator
# geolocator = Nominatim(user_agent="geoapiExercises")

# def get_coordinates(place_name):
#     try:
#         location = geolocator.geocode(place_name)
#         if location:
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except GeocoderTimedOut:
#         return get_coordinates(place_name)


# # Get coordinates
# df['latitude'], df['longitude'] = zip(*df['constituency'].apply(get_coordinates))

# # drop rows with NaN values
# df = df.dropna()
# # Display the DataFrame
# df

In [64]:
# shapely

In [65]:



unique_party = sorted(df['party'].unique())

from jinja2 import Template
import json

# Convert the DataFrame to GeoJSON format
geojson = {
    'type': 'FeatureCollection',
    'features': []
}

for _, row in df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']]
        },
        'properties': {
            'party': row['party'],
            'constituency': row['constituency'],
            'name': row['name'],
        }
    }
    geojson['features'].append(feature)

# Save the GeoJSON data to a file
with open('locations.geojson', 'w') as f:
    json.dump(geojson, f)

with open('map_template.html') as f:
    template = Template(f.read())

html = template.render(party=unique_party, geojson=geojson)

with open('map.html', 'w') as f:
    f.write(html)




In [66]:




# def generate_map_and_csv(all, df, address, radius, latitude=None, longitude=None):
#     if latitude is None or longitude is None:
#         geolocator = Nominatim(user_agent="geoapiExercises")
#         location = geolocator.geocode(address)
        
#         if location is None:
#             print(f"Error: Address '{address}' not found. Please provide latitude and longitude.")
#             return

#         latitude, longitude = location.latitude, location.longitude

#     # Filter screens within the radius
#     filtered_screens = df[df.apply(lambda row: great_circle((latitude, longitude), (row['latitude'], row['longitude'])).km <= radius, axis=1)]
#     # Concat filtered_screens to all
#     all = all.append(filtered_screens, ignore_index=True)

#     # Save the filtered screens to a CSV file
#     filtered_screens.to_csv(address + '.csv', index=False)

#     # Create a map centered on the address location
#     map_view = folium.Map(location=[latitude, longitude], zoom_start=15)

#     # Add a marker for the address location
#     folium.Marker([latitude, longitude], popup=address, icon=folium.Icon(color='red')).add_to(map_view)

#     # Add markers for the screens within the radius
#     for _, row in filtered_screens.iterrows():
#         folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='blue')).add_to(map_view)

#     # Draw a circle with the indicated radius
#     folium.Circle(
#         location=[latitude, longitude],
#         radius=radius * 1000,  # Convert the radius to meters
#         color='green',
#         fill=True,
#         fill_color='green',
#         fill_opacity=0.2
#     ).add_to(map_view)

#     # Draw a line from center to the circle
#     end_lat = latitude + (radius / 111.32)  # Assuming 111.32 km per degree of latitude
#     folium.PolyLine([(latitude, longitude), (end_lat, longitude)], color='black', weight=1).add_to(map_view)

#     # Display the radius length on the line
#     folium.Marker(
#         [(latitude + end_lat) / 2, longitude],
#         icon=DivIcon(icon_size=(150, 36), icon_anchor=(0, 0), html='<div style="font-size: 10pt; color: black;">{} km</div>'.format(radius))
#     ).add_to(map_view)

#     # Save the map as an HTML file
#     map_view.save(address + '.html')

#     return all



# #create an empty dataframe with df columns
# all = pd.DataFrame(columns=df.columns)

# all = generate_map_and_csv(all, df, "Via Giuseppe Piazzi 2-4, Milano, 20159", 2)
# all =generate_map_and_csv(all,df, "Via Crescenzo del Monte, n. 25", 2)
# all =generate_map_and_csv(all,df, "2-16 Ronda de Poniente", 10)
# all =generate_map_and_csv(all,df, "C. Marie Curie, 9, 4ª Planta, 28521 Rivas-Vaciamadrid, Madrid, Spain", 2, latitude=40.337304, longitude=-3.523987)
# all =generate_map_and_csv(all,df, "Camí de Can Camps, 17, 19, 08174 Sant Cugat del Vallès, Barcelona, Spain", 10, latitude=41.4981048, longitude=2.0729402)


In [67]:
# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.distance import great_circle
# import folium
# from folium.features import DivIcon


# def add_markers_to_map(all, df, map_view, address, radius, latitude=None, longitude=None):
#     if latitude is None or longitude is None:
#         geolocator = Nominatim(user_agent="geoapiExercises")
#         location = geolocator.geocode(address)
        
#         if location is None:
#             print(f"Error: Address '{address}' not found. Please provide latitude and longitude.")
#             return all

#         latitude, longitude = location.latitude, location.longitude

#     # Filter screens within the radius
#     filtered_screens = df[df.apply(lambda row: great_circle((latitude, longitude), (row['latitude'], row['longitude'])).km <= radius, axis=1)]
#     # Concat filtered_screens to all
#     all = all.append(filtered_screens, ignore_index=True)

#     # Save the filtered screens to a CSV file
#     filtered_screens.to_csv(address + '.csv', index=False)

#     # Add a marker for the address location
#     folium.Marker([latitude, longitude], popup=address, icon=folium.Icon(color='red')).add_to(map_view)

#     # Add markers for the screens within the radius
#     for _, row in filtered_screens.iterrows():
#         folium.Marker([row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='blue')).add_to(map_view)

#     # Draw a circle with the indicated radius
#     folium.Circle(
#         location=[latitude, longitude],
#         radius=radius * 1000,  # Convert the radius to meters
#         color='green',
#         fill=True,
#         fill_color='green',
#         fill_opacity=0.2
#     ).add_to(map_view)

#     # Draw a line from center to the circle
#     end_lat = latitude + (radius / 111.32)  # Assuming 111.32 km per degree of latitude
#     folium.PolyLine([(latitude, longitude), (end_lat, longitude)], color='black', weight=1).add_to(map_view)

#     # Display the radius length on the line
#     folium.Marker(
#         [(latitude + end_lat) / 2, longitude],
#         icon=DivIcon(icon_size=(150, 36), icon_anchor=(0, 0), html='<div style="font-size: 10pt; color: black;">{} km</div>'.format(radius))
#     ).add_to(map_view)

#     return all

# # Create an empty map
# map_view = folium.Map(zoom_start=10)
# #create an empty dataframe with df columns
# all2 = pd.DataFrame(columns=df.columns)
# # Add markers for each location
# all2 = add_markers_to_map(all2, df, map_view, "Via Giuseppe Piazzi 2-4, Milano, 20159", 1.5)
# all2 = add_markers_to_map(all2, df, map_view, "Via Crescenzo del Monte, n. 25", 2)
# all2 = add_markers_to_map(all2, df, map_view, "2-16 Ronda de Poniente", 15)
# all2 = add_markers_to_map(all2, df, map_view, "C. Marie Curie, 9, 4ª Planta, 28521 Rivas-Vaciamadrid, Madrid, Spain", 5, latitude=40.337304, longitude=-3.523987)
# all2 = add_markers_to_map(all2, df, map_view, "Camí de Can Camps, 17, 19, 08174 Sant Cugat del Vallès, Barcelona, Spain", 10, latitude=41.4981048, longitude=2.0729402)
# #all to csv
# all2.to_csv('Spain&Italy_all_locations.csv', index=False)
# # Save the map as an HTML file
# map_view.save('Spain&Italy_all_locations_map.html')

In [68]:
# all2